In [ ]:
import glob, re, os, json
from pyld import jsonld

In [58]:
ls ../


404.html
README.md*
Tables/
assets/
data_desciptors/
data_descriptors/
demo/
inputs/
obs4MIPs_frequency.json
obs4MIPs_grid_label.json
obs4MIPs_institution_id.json
obs4MIPs_license.json
obs4MIPs_nominal_resolution.json
obs4MIPs_product.json
obs4MIPs_realm.json
obs4MIPs_region.json
obs4MIPs_required_global_attributes.json
obs4MIPs_source_id.json
obs4MIPs_source_type.json
obs4MIPs_table_id.json
src/


In [ ]:
# source_type
data = json.load(open('../obs4MIPs_source_type.json'))['source_type']

for k,v in data.items():
    print(k)
    content = {
        "@context": "_context_",
        "description": v,
        "id": k.lower(),
        "name": k,
        "type": "source_type"
    }
    
    json.dump(content, open(f'../../WCRP-UNIVERSE/data_descriptors/source_type/{k.lower()}.json','w'), indent=4)

In [ ]:
# source
base = '../data_desciptors/source/'
os.system(f'mkdir -p {base}')

data = json.load(open('../obs4MIPs_source_id.json'))['source_id']

for k,v in data.items():
    print(v)
    v['id'] = k.lower()
    v['type'] = 'source'
    v['@context'] = '_context_'
    v['organisation'] = v['institution_id'].lower()
    del v['institution_id']
    v['source_type'] = v['source_type'].lower()
    v['description'] = v['source_description']
    v['activity-participation'] = 'obs4mips'
    del v['source_description']
    
    json.dump(v, open(f'{base}/{k.lower()}.json','w'), indent=4)

In [ ]:
# tables and variables
tables = glob.glob('../Tables/*.json')[1:]
tables

print(tables)

skip = False
# tbl = tables[0]
for tbl in tables:
    for c in ['formula','coordinates','grids']: 
        if c in tbl:
            skip = True
    if skip:continue
    print(tbl)
    data = json.load(open(tbl,'r'))

    t = data['Header']
    t['id'] = t['table_id'].lower().split('_')[-1]
    t['type'] = 'mip_table'
    t['@context'] = '_context_'
    t['product'] = ['observations']

    tpath = f'../../MIP-variables/data_descriptors/tables/{t["id"].lower()}.json'
    if os.path.exists(tpath):
        print(f'{tpath} exists',t['id'])
        data2 = json.load(open(tpath,'r'))
        data2['product'].extend(t['product'])
        json.dump(data2, open(tpath,'w'), indent=4)
    else:
        json.dump(t, open(tpath,'w'), indent=4)

    for vi,v in data['variable_entry'].items():
        v['id'] = vi.lower()
        v['type'] = 'mip-variable'
        v['@context'] = '_context_'
        v["mip_tables"]= [
            {
                "@id": t['id'],
                "mip-era": "obs4mips"
            }
        ]
        v['themes'] = ['TBC']
        v["modeling_realm"]= [v["modeling_realm"].lower()]
        
        vpath = f'../../MIP-variables/data_descriptors/variables/{t["id"]}.{v["id"]}.json'
        if os.path.exists(vpath):
            print(f'{vpath} exists',v['id'])
            data2 = json.load(open(vpath,'r'))
            if 'mip_tables' not in data2:
                data2['mip_tables']= []
            data2['mip_tables'].extend(v["mip_tables"])
            json.dump(data2, open(vpath,'w'), indent=4)
        else:
            json.dump(v, open(vpath,'w'), indent=4)